In [15]:
import nibabel as nib
import matplotlib.pyplot as plt

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models
from segmentation_models_pytorch.losses import DiceLoss, FocalLoss
import segmentation_models_pytorch as smp


In [17]:
# 自定义数据集类
class SpineWeb15(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        
    def __len__(self):
        return len(self.data_path)
    
    def __getitem__(self, index):
        metadata = np.load(self.data_path+str(index)+".npy")
        image = metadata[0]
        label = metadata[1]

        image_data = torch.from_numpy(image).float()
        label_data = torch.from_numpy(label).float()
        
        if self.transform:
            image_data = self.transform(image_data)
        return image_data, label_data



In [24]:
# -----------------------------
# DeepLabV3+ Train and Test Set
# -----------------------------

def calculate_iou(prediction, target):
    intersection = torch.logical_and(prediction, target).sum()
    union = torch.logical_or(prediction, target).sum()
    iou = intersection / union
    return iou.item()

def calculate_dice(prediction, target):
    intersection = torch.logical_and(prediction, target).sum()
    dice = (2 * intersection) / (prediction.sum() + target.sum())
    return dice.item()

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/train/"
test_path = "./dataset/Spineweb_dataset15/test/"

#hyper parameters set
batch_size = 32
num_epochs = 200
learning_rate = 0.00001
confidence = 0.7
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
testset = SpineWeb15(data_path=test_path, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器

model = smp.DeepLabV3Plus(encoder_name='resnet34', encoder_depth=5, encoder_weights='imagenet', encoder_output_stride=16, decoder_channels=256, decoder_atrous_rates=(12, 24, 36), in_channels=1, classes=1, activation=None, upsampling=4, aux_params=None)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss
# criterion = DiceLoss(mode="binary")   #DiceLoss
# criterion = FocalLoss(mode="binary")
criterion = smp.losses.TverskyLoss(mode="binary")

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss_train = 100.0
best_IoU_train = 0.0
best_Dice_train = 0.0

best_IoU_test = 0.0
best_Dice_test = 0.0

# 训练模型
for epoch in range(num_epochs):
    # torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0
    total_iou_test = 0
    total_dice_test = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # normalize the output
        min_value = torch.min(outputs)
        max_value = torch.max(outputs)
        outputs = (outputs - min_value) / (max_value - min_value)
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    # train set results
    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)

    # test set results
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            # squeeze the dimension that pytorch add for channels
            outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
            # normalize the output
            min_value = torch.min(outputs)
            max_value = torch.max(outputs)
            outputs = (outputs - min_value) / (max_value - min_value)
            # process the result, bigger than 0.5 is reliable
            predicted = (outputs > confidence).float()     # tensor size ([16,128,128])
            
            intersection = torch.logical_and(predicted, targets).sum((1, 2))
            union = torch.logical_or(predicted, targets).sum((1, 2))
            iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
            dice = (2 * intersection / (predicted.sum((1, 2)) + targets.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

            total_iou_test += iou.item()
            total_dice_test += dice.item()

    test_iou = total_iou_test / len(test_loader)
    test_dice = total_dice_test / len(test_loader)

    # get the best IoU and Dice
    best_loss_train = best_loss_train if best_loss_train < avg_loss else avg_loss
    best_IoU_train = best_IoU_train if best_IoU_train > avg_iou else avg_iou
    best_Dice_train = best_Dice_train if best_Dice_train > avg_dice else avg_dice
    best_IoU_test = best_IoU_test if best_IoU_test > test_iou else test_iou
    # best_Dice_test = best_Dice_test if best_Dice_test > test_dice else test_dice

    if(best_Dice_test <= test_dice):
        best_Dice_test = test_dice
        torch.save(model.state_dict(), f"./model/DeepLabV3+_checkpoint.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f},test IoU: {test_iou:.4f}, test Dice: {test_dice:.4f}")
    
print("-----------------------------------------")
print(f"Best Loss: {best_loss_train:.4f}, Best Train IoU: {best_IoU_train:.4f}, Best Train Dice: {best_Dice_train:.4f}, Best Test IoU: {best_IoU_test:.4f}, Best Test Dice: {best_Dice_test:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

Epoch [1/200], Avg Loss: 0.9456, Avg IoU: 0.0135, Avg Dice: 0.0252,test IoU: 0.0141, test Dice: 0.0248
Epoch [2/200], Avg Loss: 0.9596, Avg IoU: 0.0373, Avg Dice: 0.0657,test IoU: 0.0204, test Dice: 0.0384
Epoch [3/200], Avg Loss: 0.9610, Avg IoU: 0.0346, Avg Dice: 0.0596,test IoU: 0.0302, test Dice: 0.0560
Epoch [4/200], Avg Loss: 0.4754, Avg IoU: 0.0287, Avg Dice: 0.0476,test IoU: 0.0449, test Dice: 0.0813
Epoch [5/200], Avg Loss: 0.9404, Avg IoU: 0.0461, Avg Dice: 0.0816,test IoU: 0.0378, test Dice: 0.0698
Epoch [6/200], Avg Loss: 0.9734, Avg IoU: 0.0165, Avg Dice: 0.0284,test IoU: 0.0437, test Dice: 0.0785
Epoch [7/200], Avg Loss: 0.9497, Avg IoU: 0.0256, Avg Dice: 0.0455,test IoU: 0.0220, test Dice: 0.0414
Epoch [8/200], Avg Loss: 0.9628, Avg IoU: 0.0516, Avg Dice: 0.0852,test IoU: 0.0646, test Dice: 0.1081
Epoch [9/200], Avg Loss: 0.9465, Avg IoU: 0.0446, Avg Dice: 0.0755,test IoU: 0.0236, test Dice: 0.0427
Epoch [10/200], Avg Loss: 0.9504, Avg IoU: 0.0490, Avg Dice: 0.0857,test 

In [19]:
# -----------------------------
# DeepLabV3+ All Set
# -----------------------------

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/processed/"

#hyper parameters set
batch_size = 32
num_epochs = 200
learning_rate = 0.0001
confidence = 0.95
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path,transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = smp.DeepLabV3Plus(encoder_name='resnet34', encoder_depth=5, encoder_weights='imagenet', encoder_output_stride=16, decoder_channels=256, decoder_atrous_rates=(12, 24, 36), in_channels=1, classes=1, activation=None, upsampling=4, aux_params=None)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss
criterion = smp.losses.TverskyLoss(mode="binary")

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss = 100.0
best_IoU = 0.0
best_Dice = 0.0

# 训练模型
for epoch in range(num_epochs):
    # torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)
    
    # get the best IoU and Dice
    best_loss = best_loss if best_loss < avg_loss else avg_loss
    best_IoU = best_IoU if best_IoU > avg_iou else avg_iou
    best_Dice = best_Dice if best_Dice > avg_dice else avg_dice

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f}")
print("-----------------------------------------")
print(f"Best Loss: {best_loss:.4f}, Best IoU: {best_IoU:.4f}, Best Dice: {best_Dice:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

Epoch [1/200], Avg Loss: 0.9499, Avg IoU: 0.0077, Avg Dice: 0.0136
Epoch [2/200], Avg Loss: 0.9113, Avg IoU: 0.0753, Avg Dice: 0.1263
Epoch [3/200], Avg Loss: 0.9125, Avg IoU: 0.0663, Avg Dice: 0.1096
Epoch [4/200], Avg Loss: 0.9193, Avg IoU: 0.0659, Avg Dice: 0.1123
Epoch [5/200], Avg Loss: 0.8757, Avg IoU: 0.1112, Avg Dice: 0.1809
Epoch [6/200], Avg Loss: 0.8880, Avg IoU: 0.1061, Avg Dice: 0.1720
Epoch [7/200], Avg Loss: 0.8892, Avg IoU: 0.0799, Avg Dice: 0.1385
Epoch [8/200], Avg Loss: 0.8754, Avg IoU: 0.0903, Avg Dice: 0.1542
Epoch [9/200], Avg Loss: 0.8456, Avg IoU: 0.1226, Avg Dice: 0.1990
Epoch [10/200], Avg Loss: 0.8609, Avg IoU: 0.0930, Avg Dice: 0.1574
Epoch [11/200], Avg Loss: 0.8543, Avg IoU: 0.1119, Avg Dice: 0.1878
Epoch [12/200], Avg Loss: 0.9040, Avg IoU: 0.0684, Avg Dice: 0.1155
Epoch [13/200], Avg Loss: 0.8572, Avg IoU: 0.1032, Avg Dice: 0.1764
Epoch [14/200], Avg Loss: 0.8631, Avg IoU: 0.0946, Avg Dice: 0.1604
Epoch [15/200], Avg Loss: 0.8565, Avg IoU: 0.1230, Avg Di